In [1]:
# # -*- coding: utf-8 -*-
# """
# FastAPI + Diffusers + Cloudflared API 배포 (Colab 기반)

# - FastAPI: API 서버 구축
# - Diffusers: Stable Diffusion 모델 추론
# - Cloudflared: 외부 인터넷 노출
# - 동적 LoRA 로딩/언로딩 API 포함
# """

# # ==============================================================================
# # 0. 환경 설정 및 변수 정의
# # ==============================================================================
# import os
# import subprocess
# import json
# import time
# import threading
# import requests
# from google.colab import drive, files
# from IPython.display import display, Image as IPyImage, clear_output
# import logging

# # --- 사용자 설정 ---
# # Hugging Face Hub 또는 로컬 경로에서 불러올 Diffusers 모델 ID
# MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0"

# # (선택 사항) 초기 로드할 LoRA 가중치 파일 경로 (Google Drive 내)
# # 비워두면 초기 LoRA 없이 시작
# INITIAL_LORA_WEIGHTS_PATH = "" # 예: "/content/drive/MyDrive/Loras/YourLora/output/YourLora-000010.safetensors"

# # API 서버 포트 설정
# API_PORT = "9080"

# # --- 내부 설정 ---
# FASTAPI_APP_FILE = "/content/main.py"
# REQUIREMENTS_PATH = "/content/requirements_fastapi.txt"
# UVICORN_LOG_FILE = "/content/uvicorn.log"
# is_google_colab = 'google.colab' in str(get_ipython())
# cloudflared_process = None
# cloudflared_url = None
# uvicorn_process = None # Uvicorn 프로세스 추적용

# # 로깅 설정
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# logger = logging.getLogger(__name__)

# # ==============================================================================
# # 1. Google Drive 마운트 (LoRA 사용 시)
# # ==============================================================================
# if is_google_colab:
#     logger.info("📂 Google Drive 마운트 중...")
#     try:
#         drive.mount('/content/drive', force_remount=True)
#         logger.info("✅ Google Drive 마운트 완료.")
#         if INITIAL_LORA_WEIGHTS_PATH and not os.path.exists(INITIAL_LORA_WEIGHTS_PATH):
#              logger.error(f"초기 LoRA 파일 경로 오류: {INITIAL_LORA_WEIGHTS_PATH}")
#         elif INITIAL_LORA_WEIGHTS_PATH:
#             logger.info(f"✅ 초기 LoRA 파일 확인: {INITIAL_LORA_WEIGHTS_PATH}")
#     except Exception as e:
#         logger.error(f"Google Drive 마운트 오류: {e}")
# else:
#     logger.warning("Colab 환경 아님. Drive 마운트 생략.")

# # ==============================================================================
# # 2. 필요 패키지 설치 및 Cloudflared
# # ==============================================================================
# logger.info("\n⚙️ 필요한 패키지 설치 및 Cloudflared 준비 중...")

# # requirements_fastapi.txt 생성
# requirements_content = """
# fastapi
# uvicorn[standard] # ASGI 서버 (standard는 추가 기능 포함)
# python-multipart # FastAPI에서 폼 데이터 처리 등에 필요할 수 있음
# requests # 테스트용

# torch>=2.0.0
# # torchvision # 핸들러 직접 사용 안 함
# # torchaudio # 핸들러 직접 사용 안 함
# diffusers>=0.24.0
# transformers>=4.30.0
# accelerate>=0.20.0
# safetensors>=0.3.0
# invisible-watermark>=0.2.0
# pillow
# python-dotenv # 환경변수 관리 (선택적)
# """
# with open(REQUIREMENTS_PATH, 'w') as f: f.write(requirements_content)
# logger.info(f"✅ requirements_fastapi.txt 생성 완료: {REQUIREMENTS_PATH}")

# # pip install
# logger.info(f"   - pip install 실행: {REQUIREMENTS_PATH}")
# try:
#     # 이전 실행의 uvicorn 등 프로세스 종료 시도 (선택적)
#     # subprocess.run(['pkill', '-f', 'uvicorn'], capture_output=True)
#     pip_process = subprocess.run(['pip', 'install', '-r', REQUIREMENTS_PATH, '-qq'],
#                                  check=True, capture_output=True, text=True, encoding='utf-8')
#     logger.info("✅ Python 패키지 설치 완료.")
# except subprocess.CalledProcessError as e:
#     logger.error(f"💥 pip install 실패! (종료 코드: {e.returncode})")
#     print("--- pip stdout ---\n", e.stdout)
#     print("--- pip stderr ---\n", e.stderr)
#     raise e

# # Cloudflared 설치
# if is_google_colab:
#     logger.info("   - Cloudflared 다운로드 및 설치 중...")
#     try:
#         if os.path.exists("cloudflared-linux-amd64.deb"): os.remove("cloudflared-linux-amd64.deb")
#         subprocess.run(['wget', '-q', 'https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb'], check=True)
#         time.sleep(1)
#         subprocess.run(['dpkg', '-i', 'cloudflared-linux-amd64.deb'], check=True, capture_output=True)
#         logger.info("✅ Cloudflared 설치 완료.")
#     except Exception as e: logger.error(f"Cloudflared 설치 오류: {e}")
# else: logger.warning("Colab 환경 아님. Cloudflared 설치 생략.")

# # ==============================================================================
# # 3. FastAPI 앱 스크립트 생성 (main.py)
# # ==============================================================================
# logger.info(f"\n📝 FastAPI 앱 스크립트 생성 중 ({FASTAPI_APP_FILE})...")

# fastapi_app_content = '''
# import logging
# import os
# import base64
# import io
# import time
# import torch
# from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
# from fastapi import FastAPI, HTTPException, Request
# from pydantic import BaseModel, Field
# from PIL import Image
# import asyncio # 비동기 Lock 및 스레드 실행용
# from contextlib import asynccontextmanager
# from fastapi import FastAPI
# from fastapi.responses import JSONResponse, Response

# # --- 기본 설정 ---
# # 환경 변수 또는 기본값 사용 (스크립트 시작 시 설정된 값 활용)
# MODEL_ID = os.environ.get("TS_MODEL_ID", "stabilityai/stable-diffusion-xl-base-1.0")
# INITIAL_LORA_PATH = os.environ.get("TS_INITIAL_LORA_PATH", None)
# ADAPTER_NAME = "default" # 사용할 LoRA 어댑터 이름

# # 로깅 설정
# logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# logger = logging.getLogger("uvicorn") # Uvicorn 로거 사용 권장

# # --- 모델 및 상태 관리 ---
# # 전역 변수 대신 context manager 또는 클래스 사용 권장되나, 여기서는 간단하게 전역 사용
# pipeline = None
# current_lora_path = None
# model_lock = asyncio.Lock() # 파이프라인 접근 제어용 Lock

# # --- FastAPI 앱 생명주기 관리 (모델 로딩/언로딩) ---
# @asynccontextmanager
# async def lifespan(app: FastAPI):
#     global pipeline, current_lora_path, model_lock
#     logger.info("애플리케이션 시작: 모델 로딩 시작...")
#     start_time = time.time()
#     try:
#         if torch.cuda.is_available():
#             device = "cuda"
#             torch_dtype = torch.float16
#             logger.info(f"사용 가능 GPU 감지됨. Device: {device}, Dtype: {torch_dtype}")
#         else:
#             device = "cpu"
#             torch_dtype = torch.float32 # CPU에서는 float32 사용
#             logger.info(f"GPU 사용 불가. Device: {device}, Dtype: {torch_dtype}")

#         if torch.cuda.is_available(): logger.info(f"메모리 (로딩 전): 할당됨 {torch.cuda.memory_allocated(device)/1e9:.2f} GB, 예약됨 {torch.cuda.memory_reserved(device)/1e9:.2f} GB")

#         temp_pipeline = DiffusionPipeline.from_pretrained(
#             MODEL_ID,
#             torch_dtype=torch_dtype,
#             variant="fp16" if torch_dtype == torch.float16 else None, # GPU 사용 시 FP16 변형 사용
#             use_safetensors=True
#         )
#         temp_pipeline.scheduler = DPMSolverMultistepScheduler.from_config(temp_pipeline.scheduler.config, use_karras_sigmas=True)
#         temp_pipeline.to(device)
#         logger.info(f"모델 파이프라인 로딩 완료. Device: {temp_pipeline.device}")

#         if torch.cuda.is_available(): logger.info(f"메모리 (파이프라인 로딩 후): 할당됨 {torch.cuda.memory_allocated(device)/1e9:.2f} GB, 예약됨 {torch.cuda.memory_reserved(device)/1e9:.2f} GB")

#         pipeline = temp_pipeline # 전역 변수에 할당

#         # 초기 LoRA 로드
#         if INITIAL_LORA_PATH and os.path.exists(INITIAL_LORA_PATH):
#             logger.info(f"초기 LoRA 로딩 시도: {INITIAL_LORA_PATH}")
#             try:
#                 async with model_lock: # Lock 사용
#                     # 비동기 함수 내에서 동기 함수 실행 (모델 로딩은 블로킹 작업)
#                     await asyncio.to_thread(load_lora_blocking, INITIAL_LORA_PATH)
#                 logger.info(f"초기 LoRA 로딩 성공: {INITIAL_LORA_PATH}")
#                 current_lora_path = INITIAL_LORA_PATH
#                 if torch.cuda.is_available(): logger.info(f"메모리 (LoRA 로딩 후): 할당됨 {torch.cuda.memory_allocated(device)/1e9:.2f} GB, 예약됨 {torch.cuda.memory_reserved(device)/1e9:.2f} GB")
#             except Exception as e:
#                  logger.error(f"초기 LoRA 로딩 실패: {e}", exc_info=True)
#         elif INITIAL_LORA_PATH:
#              logger.warning(f"초기 LoRA 파일({INITIAL_LORA_PATH}) 없음.")

#         loading_time = time.time() - start_time
#         logger.info(f"모델 및 초기 LoRA 로딩 완료. (소요 시간: {loading_time:.2f}초)")

#     except Exception as e:
#         logger.error(f"모델 로딩 중 치명적 오류 발생: {e}", exc_info=True)
#         pipeline = None # 로딩 실패 시 None 유지

#     yield # 애플리케이션 실행 구간

#     # --- 애플리케이션 종료 시 ---
#     logger.info("애플리케이션 종료: 모델 정리...")
#     async with model_lock: # Lock 사용
#         pipeline = None
#         current_lora_path = None
#     if torch.cuda.is_available():
#         torch.cuda.empty_cache()
#         logger.info("GPU 캐시 정리 완료.")
#     logger.info("애플리케이션 종료 완료.")


# app = FastAPI(lifespan=lifespan)

# # --- Pydantic 모델 정의 ---
# class InferenceRequest(BaseModel):
#     prompt: str
#     negative_prompt: str = ""
#     height: int = 1024
#     width: int = 1024
#     steps: int = Field(30, gt=0, le=100) # 1 ~ 100 사이 값
#     cfg_scale: float = Field(7.5, gt=0.0, le=20.0) # 0 초과 ~ 20 이하 값
#     seed: int | None = None

# class InferenceResponse(BaseModel):
#     image_base64: str
#     generation_time_ms: int
#     model_id: str
#     current_lora: str | None

# class LoraRequest(BaseModel):
#     lora_path: str # Google Drive 내 절대 경로 등

# class StatusResponse(BaseModel):
#     status: str
#     model_id: str | None
#     device: str | None
#     current_lora_path: str | None
#     active_adapters: list | None # 활성화된 어댑터 목록 (디버깅용)

# # --- Helper Functions (Blocking tasks) ---
# # 동기 함수들은 asyncio.to_thread 로 호출해야 함

# def load_lora_blocking(lora_path: str):
#     global pipeline, current_lora_path # 전역 변수 사용 시 명시
#     if not pipeline: raise RuntimeError("파이프라인이 초기화되지 않았습니다.")
#     if not os.path.exists(lora_path): raise FileNotFoundError(f"LoRA 파일 없음: {lora_path}")

#     logger.info(f"LoRA 로딩 (동기): {lora_path}")
#     pipeline.to(pipeline.device) # 장치 확인

#     # 기존 어댑터 언로드 시도
#     try:
#         active_adapters = getattr(pipeline, "get_active_adapters", lambda: [])()
#         if ADAPTER_NAME in active_adapters:
#             logger.info(f"기존 '{ADAPTER_NAME}' 언로드 시도...")
#             if hasattr(pipeline, 'unload_lora_weights'): pipeline.unload_lora_weights(adapter_names=[ADAPTER_NAME])
#             elif hasattr(pipeline, 'delete_adapters'): pipeline.delete_adapters(ADAPTER_NAME)
#     except Exception as e: logger.warning(f"기존 LoRA 언로드 중 경고(무시): {e}")

#     logger.info(f"새 LoRA 로딩: '{lora_path}' ('{ADAPTER_NAME}')...")
#     pipeline.load_lora_weights(os.path.dirname(lora_path), weight_name=os.path.basename(lora_path), adapter_name=ADAPTER_NAME)
#     logger.info(f"LoRA 로딩 성공: {lora_path}")

#     if hasattr(pipeline, 'set_adapters'): pipeline.set_adapters([ADAPTER_NAME], adapter_weights=[1.0])
#     elif hasattr(pipeline, 'fuse_lora'): pipeline.fuse_lora(adapter_names=[ADAPTER_NAME])

#     current_lora_path = lora_path # 상태 업데이트
#     if torch.cuda.is_available(): torch.cuda.empty_cache()

# def unload_lora_blocking():
#     global pipeline, current_lora_path
#     if not pipeline: raise RuntimeError("파이프라인이 초기화되지 않았습니다.")
#     if not current_lora_path: return "현재 로드된 LoRA 없음."

#     logger.info(f"LoRA 언로드 (동기): {current_lora_path}")
#     pipeline.to(pipeline.device)
#     unloaded_path = current_lora_path
#     try:
#         if hasattr(pipeline, 'unload_lora_weights'): pipeline.unload_lora_weights(adapter_names=[ADAPTER_NAME])
#         elif hasattr(pipeline, 'delete_adapters'): pipeline.delete_adapters(ADAPTER_NAME)
#         else:
#              if hasattr(pipeline, 'set_adapters'): pipeline.set_adapters([])
#              elif hasattr(pipeline, 'unfuse_lora'): pipeline.unfuse_lora()
#         logger.info(f"성공적으로 LoRA 언로드/비활성화: {unloaded_path}")
#         current_lora_path = None
#         if torch.cuda.is_available(): torch.cuda.empty_cache()
#         return f"성공적으로 LoRA 언로드: {unloaded_path}"
#     except Exception as e:
#          logger.error(f"LoRA 언로드 중 오류: {e}", exc_info=True)
#          current_lora_path = None # 오류 시에도 상태는 초기화
#          raise RuntimeError(f"LoRA 언로드 실패: {e}") from e

# def generate_image_blocking(req: InferenceRequest) -> Image.Image:
#     global pipeline
#     if not pipeline: raise RuntimeError("파이프라인이 초기화되지 않았습니다.")

#     logger.info(f"이미지 생성 시작 (동기): prompt='{req.prompt[:50]}...'")
#     params = {
#         "negative_prompt": req.negative_prompt,
#         "num_inference_steps": req.steps,
#         "guidance_scale": req.cfg_scale,
#         "height": req.height,
#         "width": req.width,
#     }
#     if req.seed is not None:
#         params["generator"] = torch.Generator(device=pipeline.device).manual_seed(req.seed)

#     with torch.inference_mode():
#         result_image = pipeline(prompt=req.prompt, **params).images[0]
#     logger.info("이미지 생성 완료 (동기)")
#     return result_image


# # --- API 엔드포인트 ---
# @app.get("/", include_in_schema=False)
# async def root_health():
#     # 단순히 200 OK 리턴
#     return JSONResponse({"status": "ok"}, status_code=200)

# @app.get("/favicon.ico", include_in_schema=False)
# async def favicon():
#     return Response(status_code=204)

# @app.get("/status", response_model=StatusResponse)
# async def get_status():
#     """현재 서버 상태 (로드된 모델, LoRA 등) 반환"""
#     async with model_lock: # 상태 읽기 시에도 Lock 사용 (일관성)
#         if not pipeline:
#             return StatusResponse(status="error", model_id=MODEL_ID, device=None, current_lora_path=None, active_adapters=None)

#         active_adapters = []
#         try:
#             if hasattr(pipeline, 'get_active_adapters'): active_adapters = pipeline.get_active_adapters()
#             elif hasattr(pipeline, 'get_list_adapters'): active_adapters = [name for name, enabled in pipeline.get_list_adapters().items() if enabled]
#         except Exception as e: logger.warning(f"활성 어댑터 가져오기 실패: {e}")

#         return StatusResponse(
#             status="ready" if pipeline else "initializing_error",
#             model_id=MODEL_ID,
#             device=str(pipeline.device) if pipeline else None,
#             current_lora_path=current_lora_path,
#             active_adapters=active_adapters
#         )

# @app.post("/predictions", response_model=InferenceResponse)
# async def predict(request: InferenceRequest):
#     """이미지 생성 요청 처리"""
#     global pipeline
#     if not pipeline:
#         raise HTTPException(status_code=503, detail="모델 파이프라인이 준비되지 않았습니다.")

#     start_time = time.time()
#     try:
#         # 블로킹 I/O 또는 CPU/GPU 바운드 작업을 별도 스레드에서 실행
#         pil_image = await asyncio.to_thread(generate_image_blocking, request)

#         # PIL Image -> Base64
#         buffered = io.BytesIO()
#         pil_image.save(buffered, format="PNG")
#         img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

#         end_time = time.time()
#         generation_time_ms = int((end_time - start_time) * 1000)

#         # 응답 생성 전에 현재 LoRA 상태 확인 (Lock 내부에서 변경될 수 있으므로 다시 확인)
#         async with model_lock:
#             lora_in_use = current_lora_path

#         return InferenceResponse(
#             image_base64=img_str,
#             generation_time_ms=generation_time_ms,
#             model_id=MODEL_ID,
#             current_lora=lora_in_use
#         )
#     except Exception as e:
#         logger.error(f"추론 중 오류 발생: {e}", exc_info=True)
#         raise HTTPException(status_code=500, detail=f"추론 실패: {str(e)}")

# @app.post("/load-lora", status_code=200)
# async def load_lora(request: LoraRequest):
#     """지정된 경로의 LoRA 가중치를 로드"""
#     global pipeline
#     if not pipeline:
#         raise HTTPException(status_code=503, detail="모델 파이프라인이 준비되지 않았습니다.")

#     async with model_lock: # Lock을 사용하여 동시 접근 방지
#         try:
#             logger.info(f"LoRA 로드 요청 수신: {request.lora_path}")
#             # 블로킹 작업을 별도 스레드에서 실행
#             message = await asyncio.to_thread(load_lora_blocking, request.lora_path)
#             logger.info(f"LoRA 로드 요청 처리 완료.")
#             return {"status": "success", "message": message}
#         except FileNotFoundError as e:
#             logger.error(f"LoRA 파일 없음 오류: {e}")
#             raise HTTPException(status_code=404, detail=str(e))
#         except Exception as e:
#             logger.error(f"LoRA 로드 중 오류: {e}", exc_info=True)
#             raise HTTPException(status_code=500, detail=f"LoRA 로드 실패: {str(e)}")

# @app.post("/unload-lora", status_code=200)
# async def unload_lora():
#     """현재 로드된 LoRA 가중치를 언로드"""
#     global pipeline
#     if not pipeline:
#         raise HTTPException(status_code=503, detail="모델 파이프라인이 준비되지 않았습니다.")

#     async with model_lock: # Lock 사용
#         try:
#             logger.info(f"LoRA 언로드 요청 수신.")
#             # 블로킹 작업을 별도 스레드에서 실행
#             message = await asyncio.to_thread(unload_lora_blocking)
#             logger.info(f"LoRA 언로드 요청 처리 완료.")
#             return {"status": "success", "message": message}
#         except Exception as e:
#             logger.error(f"LoRA 언로드 중 오류: {e}", exc_info=True)
#             raise HTTPException(status_code=500, detail=f"LoRA 언로드 실패: {str(e)}")

# # --- 서버 실행 (Colab 스크립트에서 uvicorn으로 실행) ---
# # if __name__ == "__main__":
# #     import uvicorn
# #     # 환경 변수에서 포트 읽기 시도 (Colab에서는 스크립트에서 직접 지정)
# #     port = int(os.environ.get("PORT", 9080))
# #     uvicorn.run(app, host="0.0.0.0", port=port)
# '''
# with open(FASTAPI_APP_FILE, 'w', encoding='utf-8') as f: f.write(fastapi_app_content)
# logger.info(f"✅ FastAPI 앱 스크립트 저장 완료: {FASTAPI_APP_FILE}")

# # ==============================================================================
# # 4. FastAPI 서버 시작 (Uvicorn 사용)
# # ==============================================================================
# logger.info("\n🚀 FastAPI 서버 시작 중 (Uvicorn)...")

# # 핸들러 내부에서 사용할 환경 변수 설정 (FastAPI 앱 시작 전에 설정 필요)
# os.environ['TS_MODEL_ID'] = MODEL_ID
# if INITIAL_LORA_WEIGHTS_PATH:
#     os.environ['TS_INITIAL_LORA_PATH'] = INITIAL_LORA_WEIGHTS_PATH
# else:
#     if 'TS_INITIAL_LORA_PATH' in os.environ: del os.environ['TS_INITIAL_LORA_PATH']

# # Uvicorn 실행 명령어 (백그라운드, 로그 파일 사용)
# # --reload 옵션은 Colab에서 파일 변경 감지가 어려워 비추천
# uvicorn_cmd = [
#     "uvicorn",
#     "main:app", # FastAPI 앱 객체 위치 (main.py 파일의 app 객체)
#     "--host", "0.0.0.0",
#     "--port", API_PORT,
#     "--workers", "1" # 멀티 워커는 모델 로딩/상태 공유 문제로 1개 권장
# ]
# logger.info(f"실행할 Uvicorn 명령어: {' '.join(uvicorn_cmd)}")
# logger.info(f"Uvicorn 로그는 {UVICORN_LOG_FILE} 파일에 저장됩니다.")

# # 기존 uvicorn 프로세스 종료 시도 (선택적)
# subprocess.run(['pkill', '-f', 'uvicorn'], capture_output=True)
# time.sleep(3)

# # nohup 과 & 를 사용하여 백그라운드 실행 및 로그 리디렉션
# nohup_cmd = f"nohup {' '.join(uvicorn_cmd)} > {UVICORN_LOG_FILE} 2>&1 &"
# logger.info(f"백그라운드 실행 명령어: {nohup_cmd}")
# # shell=True 사용에 주의 필요하나, nohup/& 사용 시 불가피
# process = subprocess.Popen(nohup_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
# # Popen 자체는 즉시 반환됨. 실제 서버 시작은 백그라운드에서 진행.
# logger.info("   - Uvicorn 서버 시작 명령어 실행됨 (백그라운드).")

# # 서버 시작 대기 (모델 로딩 시간 고려)
# wait_seconds = 80 # SDXL 모델 로딩 시간 고려 (충분히 길게)
# logger.info(f"   - FastAPI 서버 초기화 및 모델 로딩 대기 중 ({wait_seconds}초)...")
# time.sleep(wait_seconds)

# # Uvicorn 로그 파일 끝부분 확인하여 시작 여부 추정
# log_check_success = False
# try:
#     if os.path.exists(UVICORN_LOG_FILE):
#         with open(UVICORN_LOG_FILE, 'r') as f:
#             lines = f.readlines()
#             tail_lines = lines[-20:] # 마지막 20줄 확인
#             print("\n--- Uvicorn 로그 (마지막 20줄) ---")
#             for line in tail_lines:
#                 print(line.strip())
#                 # Uvicorn 시작 성공 메시지 확인 (버전마다 다를 수 있음)
#                 if f"Uvicorn running on http://0.0.0.0:{API_PORT}" in line:
#                     logger.info("✅ Uvicorn 로그에서 서버 시작 메시지 확인됨.")
#                     log_check_success = True
#             if not log_check_success and lines:
#                 logger.warning("⚠️ Uvicorn 로그에서 명확한 시작 메시지를 찾지 못함. 상태 엔드포인트로 확인 필요.")
#             elif not lines:
#                  logger.warning("⚠️ Uvicorn 로그 파일이 비어 있습니다.")
#     else:
#         logger.warning(f"⚠️ Uvicorn 로그 파일({UVICORN_LOG_FILE})이 생성되지 않았습니다.")
# except Exception as e:
#     logger.error(f"Uvicorn 로그 확인 중 오류: {e}")

# # 최종 확인: 간단한 상태 요청 보내보기
# if log_check_success: # 로그에서 시작 확인 시에만 시도
#     logger.info("   - 서버 상태 확인 시도 (GET /status)...")
#     try:
#         status_url = f"http://127.0.0.1:{API_PORT}/status"
#         response = requests.get(status_url, timeout=30)
#         response.raise_for_status()
#         status_data = response.json()
#         if status_data.get("status") == "ready":
#             logger.info("✅ FastAPI 서버 상태 'ready' 확인!")
#         else:
#             logger.warning(f"⚠️ FastAPI 서버 상태가 'ready'가 아님: {status_data}")
#     except Exception as e:
#         logger.error(f"❌ FastAPI 서버 상태 확인 실패: {e}")
#         logger.error("   - 서버가 완전히 시작되지 않았거나 문제가 발생했을 수 있습니다.")
#         logger.error(f"   - Uvicorn 로그 파일({UVICORN_LOG_FILE})을 확인하세요.")
#         # 문제가 심각하면 여기서 중단
#         raise RuntimeError("FastAPI 서버 시작 또는 상태 확인 실패")
# else:
#      logger.error("❌ Uvicorn 로그에서 서버 시작을 확인할 수 없어 중단합니다.")
#      raise RuntimeError("FastAPI 서버 시작 확인 실패")


Mounted at /content/drive

--- Uvicorn 로그 (마지막 20줄) ---
E0000 00:00:1744958119.814446   31475 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744958119.820939   31475 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 06:35:19.842455: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:     Started server process [31475]
INFO:     Waiting for application startup.
INFO:     애플리케이션 시작: 모델 로딩 시작...
INFO:     사용 가능 GPU 감지됨. Device: cuda, Dtype: torch.float16
INFO:     메모리 (로딩 전): 할당됨 0.00 GB, 예약됨 0.00 GB

Loading pipeline components...: 10

In [12]:
# -*- coding: utf-8 -*-
"""
FastAPI + Diffusers + ngrok API 배포 최종 스크립트 (Colab 기반)

- FastAPI: API 서버 구축
- Diffusers: Stable Diffusion 모델 추론
- ngrok: 외부 인터넷 노출 (Authtoken 필요)
- 동적 LoRA 로딩/언로딩 API 포함
- TorchServe 관련 코드 모두 제거
- 기본 포트 9080 사용
"""

# ==============================================================================
# 0. 환경 설정 및 변수 정의
# ==============================================================================
import os
import subprocess
import json
import time
import threading
import requests
from google.colab import drive, files # LoRA 사용 시 필요할 수 있음
from IPython.display import display, Image as IPyImage, clear_output
import base64
import io
from PIL import Image
import logging
import shlex # 명령어 안전하게 분리하기 위해 사용

# --- 사용자 설정 ---
# @markdown ### 모델 및 LoRA 설정
# @markdown 사용할 Hugging Face Diffusers 모델 ID 또는 경로
MODEL_ID = "stabilityai/stable-diffusion-xl-base-1.0" # @param {type:"string"}
# @markdown (선택 사항) 시작 시 로드할 LoRA 파일 경로 (Google Drive 내). 비워두면 LoRA 없이 시작.
INITIAL_LORA_WEIGHTS_PATH = "" # @param {type:"string"}

# @markdown ---
# @markdown ### ngrok 설정
# @markdown [ngrok 대시보드](https://dashboard.ngrok.com/get-started/your-authtoken)에서 Authtoken을 복사하여 붙여넣으세요.
# @markdown **주의:** Authtoken은 비밀 정보이므로 노트북 공유 시 각별히 주의하세요.
NGROK_AUTH_TOKEN = "2vtLqb9HoP0xoiiwqovFN2laGrd_6pD67sTGUxgGasqEDTwHb"  # @param {type:"string"}

# --- 내부 설정 ---
FASTAPI_APP_FILE = "/content/main.py" # 생성될 FastAPI 앱 파일
REQUIREMENTS_PATH = "/content/requirements_fastapi_ngrok.txt"
UVICORN_LOG_FILE = "/content/uvicorn.log" # Uvicorn 로그 파일 경로
API_PORT = "9080" # FastAPI 서버가 리스닝할 포트
MODEL_API_NAME = "sdxl_diffusers" # API 경로 등에 참고용 (현재 직접 사용 안 함)
is_google_colab = 'google.colab' in str(get_ipython())
public_url = None # ngrok URL 저장 변수
uvicorn_process = None # Uvicorn 프로세스 추적용

# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


In [17]:

# ==============================================================================
# 1. Google Drive 마운트 (LoRA 사용 시)
# ==============================================================================
if INITIAL_LORA_WEIGHTS_PATH and is_google_colab: # LoRA 경로가 있고 Colab일 때만 마운트
    logger.info("📂 Google Drive 마운트 중...")
    try:
        drive.mount('/content/drive', force_remount=True)
        logger.info("✅ Google Drive 마운트 완료.")
        # 초기 LoRA 파일 존재 확인
        if not os.path.exists(INITIAL_LORA_WEIGHTS_PATH):
             # 치명적 오류 대신 경고 표시
             logger.error(f"⚠️ 지정된 초기 LoRA 파일 경로를 찾을 수 없습니다: {INITIAL_LORA_WEIGHTS_PATH}")
             INITIAL_LORA_WEIGHTS_PATH = "" # 경로 무효화
        else:
             logger.info(f"✅ 초기 LoRA 파일 확인: {INITIAL_LORA_WEIGHTS_PATH}")
    except Exception as e:
        logger.error(f"Google Drive 마운트 중 오류 발생: {e}")
        INITIAL_LORA_WEIGHTS_PATH = "" # 오류 시 경로 무효화
elif INITIAL_LORA_WEIGHTS_PATH and not is_google_colab:
     logger.warning("Colab 환경 아님. Drive 마운트 생략. 로컬 경로 유효성 확인 필요.")
     if not os.path.exists(INITIAL_LORA_WEIGHTS_PATH):
         logger.error(f"⚠️ 초기 LoRA 파일 경로 오류 (로컬): {INITIAL_LORA_WEIGHTS_PATH}")
         INITIAL_LORA_WEIGHTS_PATH = ""
else:
     logger.info("초기 LoRA 경로가 설정되지 않아 Drive 마운트를 건너<0xEB><0x9C><0x8D>니다.")

# ==============================================================================
# 2. 필요 패키지 설치 (pyngrok 추가)
# ==============================================================================
logger.info("\n⚙️ 필요한 패키지 설치 중 (pyngrok 포함)...")

# requirements_fastapi_ngrok.txt 생성
requirements_content = """
fastapi>=0.100.0 # 버전 명시 권장
uvicorn[standard]>=0.20.0 # ASGI 서버
python-multipart # FastAPI 폼 데이터 처리
requests # API 테스트용
pyngrok>=7.0.0 # ngrok 사용 라이브러리

torch>=2.0.0
diffusers>=0.24.0 # LoRA API 안정성 고려
transformers>=4.30.0
accelerate>=0.20.0
safetensors>=0.3.0
invisible-watermark>=0.2.0
pillow
python-dotenv # .env 파일 로딩 (선택적)
"""
with open(REQUIREMENTS_PATH, 'w') as f: f.write(requirements_content)
logger.info(f"✅ requirements_fastapi_ngrok.txt 생성 완료: {REQUIREMENTS_PATH}")

# pip install
logger.info(f"   - pip install 실행: {REQUIREMENTS_PATH}")
try:
    # 이전 실행의 uvicorn 등 프로세스 종료 시도 (선택적)
    subprocess.run(['pkill', '-f', 'uvicorn'], capture_output=True)
    pip_process = subprocess.run(['pip', 'install', '-r', REQUIREMENTS_PATH, '-qq'],
                                 check=True, capture_output=True, text=True, encoding='utf-8')
    logger.info("✅ Python 패키지 설치 완료.")
except subprocess.CalledProcessError as e:
    logger.error(f"💥 pip install 실패! (종료 코드: {e.returncode})")
    print("--- pip stdout ---\n", e.stdout)
    print("--- pip stderr ---\n", e.stderr)
    # pip 설치 실패는 치명적이므로 스크립트 중단
    raise RuntimeError(f"패키지 설치 실패: {e}")

# ngrok Authtoken 확인
if not NGROK_AUTH_TOKEN:
    logger.warning("⚠️ ngrok Authtoken이 입력되지 않았습니다! ngrok 터널 생성에 실패하거나 제한이 있을 수 있습니다.")
    # Authtoken 없이 진행은 가능하나, 경고 표시
else:
     logger.info("✅ ngrok Authtoken 확인됨 (입력값 기준).")


# ==============================================================================
# 3. FastAPI 앱 스크립트 생성 (main.py)
# ==============================================================================
logger.info(f"\n📝 FastAPI 앱 스크립트 생성 중 ({FASTAPI_APP_FILE})...")

# FastAPI 앱 코드 (f''' 대신 ''' 사용, APP_ 환경변수 사용)
fastapi_app_content = '''
import logging
import os
import base64
import io
import time
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from fastapi import FastAPI, HTTPException, Request
from pydantic import BaseModel, Field
from PIL import Image
import asyncio
from contextlib import asynccontextmanager
import threading # 전역 변수 보호용 Lock (asyncio.Lock도 가능)

# --- 기본 설정 ---
# 환경 변수 또는 기본값 사용
APP_MODEL_ID = os.environ.get("APP_MODEL_ID", "stabilityai/stable-diffusion-xl-base-1.0")
APP_INITIAL_LORA_PATH = os.environ.get("APP_INITIAL_LORA_PATH", None)
ADAPTER_NAME = "default" # 사용할 LoRA 어댑터 이름

logging.basicConfig(level=logging.INFO, format='%(asctime)s - [%(levelname)s] - %(name)s - %(message)s')
logger = logging.getLogger("uvicorn.error") # Uvicorn 에러 로거에 출력

# --- 모델 및 상태 관리 ---
pipeline = None
current_lora_path = None
model_lock = threading.Lock() # 동기 함수에서 전역 변수 보호용

# --- FastAPI 앱 생명주기 관리 (모델 로딩/언로딩) ---
@asynccontextmanager
async def lifespan(app: FastAPI):
    global pipeline, current_lora_path, model_lock
    logger.info("Lifespan: 애플리케이션 시작 - 모델 로딩...")
    start_time = time.time()
    try:
        # 모델 로딩 로직 (동기) - lifespan은 동기 함수 실행 가능
        device = "cuda" if torch.cuda.is_available() else "cpu"
        torch_dtype = torch.float16 if device == "cuda" else torch.float32
        logger.info(f"Lifespan: Device={device}, Dtype={torch_dtype}")

        if torch.cuda.is_available(): logger.info(f"Lifespan: 메모리 (로딩 전) Allocated={torch.cuda.memory_allocated(device)/1e9:.2f}GB, Reserved={torch.cuda.memory_reserved(device)/1e9:.2f}GB")

        temp_pipeline = DiffusionPipeline.from_pretrained(
            APP_MODEL_ID,
            torch_dtype=torch_dtype,
            variant="fp16" if torch_dtype == torch.float16 else None,
            use_safetensors=True
        )
        temp_pipeline.scheduler = DPMSolverMultistepScheduler.from_config(temp_pipeline.scheduler.config, use_karras_sigmas=True)
        temp_pipeline.to(device)
        logger.info(f"Lifespan: 모델 파이프라인 로딩 완료. Device={temp_pipeline.device}")
        if torch.cuda.is_available(): logger.info(f"Lifespan: 메모리 (파이프라인 로딩 후) Allocated={torch.cuda.memory_allocated(device)/1e9:.2f}GB, Reserved={torch.cuda.memory_reserved(device)/1e9:.2f}GB")

        # 전역 변수 할당 (Lock 불필요, 시작 시 한 번만 실행됨)
        pipeline = temp_pipeline

        # 초기 LoRA 로드 (동기)
        if APP_INITIAL_LORA_PATH and os.path.exists(APP_INITIAL_LORA_PATH):
            logger.info(f"Lifespan: 초기 LoRA 로딩 시도: {APP_INITIAL_LORA_PATH}")
            try:
                # lifespan 내에서는 Lock 없이 직접 호출 가능 (시작 시 동기 실행)
                load_lora_blocking(APP_INITIAL_LORA_PATH)
                logger.info(f"Lifespan: 초기 LoRA 로딩 성공: {APP_INITIAL_LORA_PATH}")
                # current_lora_path 는 load_lora_blocking 내부에서 업데이트됨
                if torch.cuda.is_available(): logger.info(f"Lifespan: 메모리 (LoRA 로딩 후) Allocated={torch.cuda.memory_allocated(device)/1e9:.2f}GB, Reserved={torch.cuda.memory_reserved(device)/1e9:.2f}GB")
            except Exception as e:
                 logger.error(f"Lifespan: 초기 LoRA 로딩 실패: {e}", exc_info=True)
                 # 실패해도 서버는 시작될 수 있음
        elif APP_INITIAL_LORA_PATH:
             logger.warning(f"Lifespan: 초기 LoRA 파일({APP_INITIAL_LORA_PATH}) 없음.")

        loading_time = time.time() - start_time
        logger.info(f"Lifespan: 모델 및 초기 LoRA 로딩 완료. (소요 시간: {loading_time:.2f}초)")

    except Exception as e:
        logger.error(f"Lifespan: 모델 로딩 중 치명적 오류 발생: {e}", exc_info=True)
        pipeline = None # 로딩 실패 시 None 유지

    yield # 애플리케이션 실행 구간

    # --- 애플리케이션 종료 시 ---
    logger.info("Lifespan: 애플리케이션 종료 - 모델 정리...")
    with model_lock: # Lock 사용하여 안전하게 정리
        pipeline = None
        current_lora_path = None
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        logger.info("Lifespan: GPU 캐시 정리 완료.")
    logger.info("Lifespan: 애플리케이션 종료 완료.")


app = FastAPI(lifespan=lifespan)

# --- Pydantic 모델 정의 ---
class InferenceRequest(BaseModel):
    prompt: str
    negative_prompt: str = ""
    height: int = Field(1024, gt=0)
    width: int = Field(1024, gt=0)
    steps: int = Field(30, gt=0, le=100)
    cfg_scale: float = Field(7.5, gt=0.0, le=20.0)
    seed: int | None = None

class InferenceResponse(BaseModel):
    image_base64: str
    generation_time_ms: int
    model_id: str
    current_lora: str | None

class LoraRequest(BaseModel):
    lora_path: str

class StatusResponse(BaseModel):
    status: str
    model_id: str | None
    device: str | None
    current_lora_path: str | None
    active_adapters: list | None


# --- Helper Functions (Blocking tasks) ---
# 이 함수들은 전역 pipeline과 current_lora_path를 직접 수정하므로 Lock 필요

def load_lora_blocking(lora_path: str) -> str:
    global pipeline, current_lora_path # 전역 변수 사용 명시
    if not pipeline: raise RuntimeError("파이프라인이 초기화되지 않았습니다.")
    if not os.path.exists(lora_path): raise FileNotFoundError(f"LoRA 파일 없음: {lora_path}")

    logger.info(f"LoRA 로딩 (동기): {lora_path}")
    pipeline.to(pipeline.device) # 장치 확인

    # 기존 어댑터 언로드 시도
    try:
        active_adapters = []
        # Diffusers 버전에 따른 API 호환성 고려
        if hasattr(pipeline, 'get_active_adapters'):
             active_adapters = pipeline.get_active_adapters()
        elif hasattr(pipeline, 'get_list_adapters'):
             adapter_info = pipeline.get_list_adapters()
             active_adapters = [name for name, enabled in adapter_info.items() if enabled]

        if ADAPTER_NAME in active_adapters:
            logger.info(f"기존 '{ADAPTER_NAME}' 언로드/삭제 시도...")
            # 최신 버전 우선 (delete_adapters 가 unload 포함 가능성)
            if hasattr(pipeline, 'delete_adapters'):
                 pipeline.delete_adapters(ADAPTER_NAME)
                 logger.info("delete_adapters 호출 완료.")
            elif hasattr(pipeline, 'unload_lora_weights'):
                 pipeline.unload_lora_weights(adapter_names=[ADAPTER_NAME])
                 logger.info("unload_lora_weights 호출 완료.")
            else:
                 logger.warning("LoRA 언로드/삭제 메서드를 찾을 수 없습니다.")
    except Exception as e:
        logger.warning(f"기존 LoRA 언로드 중 경고(무시): {e}")

    logger.info(f"새 LoRA 로딩: '{lora_path}' ('{ADAPTER_NAME}')...")
    pipeline.load_lora_weights(
        os.path.dirname(lora_path),
        weight_name=os.path.basename(lora_path),
        adapter_name=ADAPTER_NAME
    )
    logger.info(f"LoRA 가중치 로딩 성공: {lora_path}")

    # 활성화 (set_adapters 사용 권장)
    if hasattr(pipeline, 'set_adapters'):
        pipeline.set_adapters([ADAPTER_NAME], adapter_weights=[1.0])
        logger.info(f"'{ADAPTER_NAME}' 활성화 (set_adapters)")
    elif hasattr(pipeline, 'fuse_lora'): # 구버전 호환성
         logger.warning("set_adapters 를 찾을 수 없음. fuse_lora() 시도.")
         pipeline.fuse_lora(adapter_names=[ADAPTER_NAME])

    current_lora_path = lora_path # 상태 업데이트
    if torch.cuda.is_available(): torch.cuda.empty_cache()
    return f"성공적으로 LoRA 로드: {lora_path}"

def unload_lora_blocking() -> str:
    global pipeline, current_lora_path
    if not pipeline: raise RuntimeError("파이프라인이 초기화되지 않았습니다.")
    if not current_lora_path: return "현재 로드된 LoRA 없음."

    logger.info(f"LoRA 언로드 (동기): {current_lora_path}")
    pipeline.to(pipeline.device)
    unloaded_path = current_lora_path
    try:
        logger.info(f"'{ADAPTER_NAME}' 언로드/삭제 시도...")
        if hasattr(pipeline, 'delete_adapters'):
             pipeline.delete_adapters(ADAPTER_NAME)
             logger.info("delete_adapters 호출 완료.")
        elif hasattr(pipeline, 'unload_lora_weights'):
             pipeline.unload_lora_weights(adapter_names=[ADAPTER_NAME])
             logger.info("unload_lora_weights 호출 완료.")
        else: # Fallback 비활성화
             if hasattr(pipeline, 'set_adapters'): pipeline.set_adapters([]); logger.info("set_adapters([]) 호출")
             elif hasattr(pipeline, 'unfuse_lora'): pipeline.unfuse_lora(); logger.info("unfuse_lora() 호출")
             else: logger.warning("LoRA 비활성화 메서드 찾을 수 없음")

        logger.info(f"성공적으로 LoRA 언로드/비활성화: {unloaded_path}")
        current_lora_path = None # 상태 업데이트
        if torch.cuda.is_available(): torch.cuda.empty_cache()
        return f"성공적으로 LoRA 언로드: {unloaded_path}"
    except Exception as e:
         logger.error(f"LoRA 언로드 중 오류: {e}", exc_info=True)
         current_lora_path = None # 오류 시에도 상태 초기화
         raise RuntimeError(f"LoRA 언로드 실패: {e}") from e

def generate_image_blocking(req: InferenceRequest) -> Image.Image:
    global pipeline # pipeline은 읽기만 하므로 Lock 불필요
    if not pipeline: raise RuntimeError("파이프라인이 초기화되지 않았습니다.")

    logger.info(f"이미지 생성 시작 (동기): prompt='{req.prompt[:50]}...'")
    start_gen_time = time.time()
    # 요청 파라미터 준비
    params = {
        "negative_prompt": req.negative_prompt,
        "num_inference_steps": req.steps,
        "guidance_scale": req.cfg_scale,
        "height": req.height,
        "width": req.width,
    }
    if req.seed is not None:
        # Generator는 매번 생성하거나, 시드별로 캐싱할 수 있음
        params["generator"] = torch.Generator(device=pipeline.device).manual_seed(req.seed)

    # 추론 실행
    with torch.inference_mode():
        result_image = pipeline(prompt=req.prompt, **params).images[0]

    logger.info(f"이미지 생성 완료 (동기, {(time.time() - start_gen_time)*1000:.0f}ms)")
    return result_image


# --- API 엔드포인트 ---

@app.get("/status", response_model=StatusResponse)
async def get_status():
    """현재 서버 상태 (로드된 모델, LoRA 등) 반환"""
    with model_lock: # 상태 읽기 시에도 Lock 사용 (current_lora_path 일관성)
        if not pipeline:
            status = "error"
            device = None
            lora = current_lora_path # pipeline 없어도 lora 경로 변수 확인 가능
            adapters = None
        else:
            status = "ready"
            device = str(pipeline.device)
            lora = current_lora_path
            adapters = []
            try:
                if hasattr(pipeline, 'get_active_adapters'): adapters = pipeline.get_active_adapters()
                elif hasattr(pipeline, 'get_list_adapters'): adapters = [name for name, enabled in pipeline.get_list_adapters().items() if enabled]
            except Exception as e: logger.warning(f"활성 어댑터 가져오기 실패: {e}")

        return StatusResponse(
            status=status,
            model_id=APP_MODEL_ID,
            device=device,
            current_lora_path=lora,
            active_adapters=adapters
        )

@app.post("/predictions", response_model=InferenceResponse)
async def predict(request: InferenceRequest):
    """이미지 생성 요청 처리"""
    if not pipeline:
        raise HTTPException(status_code=503, detail="모델 파이프라인 준비되지 않음.")

    start_time = time.time()
    try:
        # 블로킹 작업을 별도 스레드에서 실행
        pil_image = await asyncio.to_thread(generate_image_blocking, request)

        # PIL Image -> Base64 (이 작업도 오래 걸리면 to_thread 고려)
        buffered = io.BytesIO()
        pil_image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

        end_time = time.time()
        generation_time_ms = int((end_time - start_time) * 1000)

        # 응답 생성 전 현재 LoRA 상태 확인 (Lock 내부에서 변경될 수 있으므로 다시 확인)
        with model_lock:
            lora_in_use = current_lora_path

        return InferenceResponse(
            image_base64=img_str,
            generation_time_ms=generation_time_ms,
            model_id=APP_MODEL_ID,
            current_lora=lora_in_use
        )
    # 특정 사용자 입력 오류 처리 (예: Pydantic 검증 실패는 FastAPI가 처리)
    # except ValueError as ve:
    #     raise HTTPException(status_code=400, detail=str(ve))
    except Exception as e:
        logger.error(f"추론 중 오류 발생: {e}", exc_info=True)
        raise HTTPException(status_code=500, detail=f"추론 실패: {str(e)}")

@app.post("/load-lora", status_code=200)
async def load_lora(request: LoraRequest):
    """지정된 경로의 LoRA 가중치를 로드"""
    if not pipeline:
        raise HTTPException(status_code=503, detail="모델 파이프라인 준비되지 않음.")

    # Lock을 사용하여 동시 접근 방지
    # asyncio.Lock 은 async 함수 내에서 사용해야 함
    # 여기서는 model_lock (threading.Lock)을 사용하고 동기 함수를 스레드에서 실행
    try:
        logger.info(f"LoRA 로드 요청 수신: {request.lora_path}")
        # 블로킹 함수 load_lora_blocking 은 내부에서 model_lock 사용
        message = await asyncio.to_thread(load_lora_blocking, request.lora_path)
        logger.info(f"LoRA 로드 요청 처리 완료.")
        return {"status": "success", "message": message}
    except FileNotFoundError as e:
        logger.error(f"LoRA 파일 없음 오류: {e}")
        raise HTTPException(status_code=404, detail=str(e))
    except Exception as e:
        logger.error(f"LoRA 로드 중 오류: {e}", exc_info=True)
        raise HTTPException(status_code=500, detail=f"LoRA 로드 실패: {str(e)}")

@app.post("/unload-lora", status_code=200)
async def unload_lora():
    """현재 로드된 LoRA 가중치를 언로드"""
    if not pipeline:
        raise HTTPException(status_code=503, detail="모델 파이프라인 준비되지 않음.")

    try:
        logger.info(f"LoRA 언로드 요청 수신.")
        # 블로킹 함수 unload_lora_blocking 은 내부에서 model_lock 사용
        message = await asyncio.to_thread(unload_lora_blocking)
        logger.info(f"LoRA 언로드 요청 처리 완료.")
        return {"status": "success", "message": message}
    except Exception as e:
        logger.error(f"LoRA 언로드 중 오류: {e}", exc_info=True)
        raise HTTPException(status_code=500, detail=f"LoRA 언로드 실패: {str(e)}")

# uvicorn으로 실행 시 이 파일이 직접 실행되지는 않음
# if __name__ == "__main__":
#     print("이 파일은 uvicorn을 통해 실행되어야 합니다: uvicorn main:app --host 0.0.0.0 --port 9080")

'''
with open(FASTAPI_APP_FILE, 'w', encoding='utf-8') as f: f.write(fastapi_app_content)
logger.info(f"✅ FastAPI 앱 스크립트 저장 완료: {FASTAPI_APP_FILE}")


# ==============================================================================
# 4. FastAPI 서버 시작 (Uvicorn 사용)
# ==============================================================================
logger.info("\n🚀 FastAPI 서버 시작 중 (Uvicorn)...")

# FastAPI 앱 내부에서 사용할 환경 변수 설정
os.environ['APP_MODEL_ID'] = MODEL_ID
if INITIAL_LORA_WEIGHTS_PATH: os.environ['APP_INITIAL_LORA_PATH'] = INITIAL_LORA_WEIGHTS_PATH
else:
    if 'APP_INITIAL_LORA_PATH' in os.environ: del os.environ['APP_INITIAL_LORA_PATH']

# Uvicorn 실행 명령어
uvicorn_cmd_parts = ["uvicorn", "main:app", "--host", "0.0.0.0", "--port", API_PORT, "--workers", "1"]
logger.info(f"실행할 Uvicorn 명령어: {' '.join(uvicorn_cmd_parts)}")
logger.info(f"Uvicorn 로그는 '{UVICORN_LOG_FILE}' 파일에 저장됩니다.")

# 기존 uvicorn 프로세스 종료
subprocess.run(['pkill', '-f', 'uvicorn'], capture_output=True); time.sleep(3)

# nohup으로 백그라운드 실행 및 로그 리디렉션
# shell=True 대신 shlex.split 사용 시도 (더 안전) -> nohup/& 처리 불가
# nohup_cmd = f"nohup {' '.join(uvicorn_cmd_parts)} > {UVICORN_LOG_FILE} 2>&1 &" # 간단하게 shell=True 사용
cmd_str = f"nohup {' '.join(uvicorn_cmd_parts)} > {UVICORN_LOG_FILE} 2>&1 & disown" # disown 추가 시도
logger.info(f"백그라운드 실행 명령어: {cmd_str}")
# Popen 대신 run 사용하고 & 로 백그라운드 실행 (더 간단할 수 있음)
# subprocess.run(cmd_str, shell=True)
uvicorn_process = subprocess.Popen(cmd_str, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
# Popen 은 명령어 실행 자체의 성공 여부만 판단, 백그라운드 프로세스 상태는 별도 확인 필요
logger.info("   - Uvicorn 서버 시작 명령어 실행됨 (백그라운드).")

# 서버 시작 및 모델 로딩 대기 (시간 충분히)
wait_seconds = 150 # SDXL + 초기 LoRA 로딩 시간 고려 (넉넉하게)
logger.info(f"   - FastAPI 서버 초기화 및 모델 로딩 대기 중 ({wait_seconds}초)...")
# 대기 시간 동안 로그 파일 변화를 감지하거나, 주기적으로 상태 체크
time.sleep(wait_seconds) # 일단 고정 시간 대기

# Uvicorn 로그 및 상태 확인
server_ready = False
logger.info(f"   - {wait_seconds}초 경과. 로그 및 상태 확인 시작...")
try:
    if os.path.exists(UVICORN_LOG_FILE):
        with open(UVICORN_LOG_FILE, 'r', encoding='utf-8', errors='replace') as f: lines = f.readlines(); tail_lines = lines[-20:]
        print("\n--- Uvicorn 로그 (마지막 20줄) ---"); [print(line.strip()) for line in tail_lines]
        # 실제 시작 완료 메시지 확인 (uvicorn 버전에 따라 다를 수 있음)
        if any(f"Application startup complete." in line for line in lines) or \
           any(f"Uvicorn running on http://0.0.0.0:{API_PORT}" in line for line in lines):
            logger.info("✅ Uvicorn 로그에서 서버 시작/준비 메시지 확인됨.")
            server_ready = True
        else: logger.warning("⚠️ Uvicorn 로그에서 명확한 시작/준비 메시지를 찾지 못함.")
    else: logger.warning(f"⚠️ Uvicorn 로그 파일({UVICORN_LOG_FILE}) 없음.")

    if server_ready: # 로그에서 시작했으면 상태 엔드포인트로 최종 확인
        logger.info("   - 서버 상태 확인 시도 (GET /status)...")
        status_url = f"http://127.0.0.1:{API_PORT}/status"
        response = requests.get(status_url, timeout=30)
        response.raise_for_status(); status_data = response.json()
        if status_data.get("status") == "ready": logger.info("✅ FastAPI 서버 상태 'ready' 최종 확인!")
        else: logger.warning(f"⚠️ FastAPI 서버 상태가 'ready'가 아님: {status_data}"); server_ready = False
    else: logger.error("❌ Uvicorn 로그에서 서버 시작을 확인할 수 없음.")

except Exception as e:
    logger.error(f"❌ FastAPI 서버 상태 확인 실패: {e}")
    logger.error(f"   - Uvicorn 로그 파일({UVICORN_LOG_FILE})을 직접 확인하세요.")
    server_ready = False

if not server_ready:
    # 로그 파일 내용 전체 출력 (디버깅 도움)
    if os.path.exists(UVICORN_LOG_FILE):
        logger.error("--- 전체 Uvicorn 로그 ---")
        try:
            with open(UVICORN_LOG_FILE, 'r', encoding='utf-8', errors='replace') as f: print(f.read())
        except Exception as read_e: print(f"로그 파일 읽기 오류: {read_e}")
        logger.error("-----------------------")
    raise RuntimeError("FastAPI 서버 시작 또는 상태 확인 실패")


# ==============================================================================
# 5. ngrok 터널 시작
# ==============================================================================
# FastAPI 서버가 준비된 경우에만 실행
if server_ready and is_google_colab:
    logger.info("\n☁️ ngrok 터널 시작 중...")
    from pyngrok import ngrok, conf

    if not NGROK_AUTH_TOKEN:
        logger.error("❌ ngrok Authtoken이 없습니다! https://dashboard.ngrok.com/get-started/your-authtoken 에서 확인 후 입력하세요.")
        raise ValueError("ngrok Authtoken 필요")
    else:
        try:
            # 기존 ngrok 터널/프로세스 정리
            try:
                for tunnel in ngrok.get_tunnels(): ngrok.disconnect(tunnel.public_url); logger.info(f"   - 기존 ngrok 터널 종료: {tunnel.public_url}")
                ngrok.kill()
                time.sleep(2)
            except Exception as ng_kill_e: logger.warning(f"기존 ngrok 종료 중 오류(무시): {ng_kill_e}")

            # Authtoken 설정
            ngrok.set_auth_token(NGROK_AUTH_TOKEN)
            logger.info("   - ngrok Authtoken 설정 완료.")

            # 터널 생성
            logger.info(f"   - 포트 {API_PORT}에 대한 ngrok 터널 생성 시도...")
            # Colab 환경에 맞는 설정 추가 (선택적)
            conf.get_default().region = 'ap' # 아시아 태평양 지역 서버 사용 시도 (ap, eu, au, sa, jp, in)
            # conf.get_default().keep_log_files = True # 로그 파일 유지 (디버깅용)

            public_url = ngrok.connect(API_PORT, name=f"fastapi-colab-{time.strftime('%Y%m%d%H%M%S')}") # 고유 이름 부여 시도
            logger.info(f"✅ ngrok 터널 URL: {public_url}")

        except Exception as e:
            logger.error(f"❌ ngrok 터널 생성 실패: {e}", exc_info=True) # 상세 오류 출력
            public_url = None # 실패 시 URL 초기화
elif not is_google_colab:
    logger.info(f"\n☁️ 로컬 접속 URL: http://127.0.0.1:{API_PORT}")


# ==============================================================================
# 6. API 테스트
# ==============================================================================
# 테스트는 URL 이 성공적으로 생성되었을 때만 의미 있음
if public_url or (not is_google_colab and server_ready):
    logger.info("\n🧪 API 추론 및 관리 테스트 시작...")
    target_base_url = public_url if public_url else f"http://127.0.0.1:{API_PORT}"
    status_url = f"{target_base_url}/status"
    predictions_url = f"{target_base_url}/predictions"
    load_lora_url = f"{target_base_url}/load-lora"
    unload_lora_url = f"{target_base_url}/unload-lora"
    logger.info(f"   - 테스트 대상 URL: {target_base_url}")

    # --- 테스트 1: 상태 조회 ---
    logger.info("\n   --- 테스트 1: 상태 조회 ---")
    try:
        response = requests.get(status_url, timeout=30)
        response.raise_for_status(); logger.info(f"   - 상태 조회 성공: {response.json()}")
    except Exception as e: logger.error(f"   - 상태 조회 실패: {e}")

    # --- 테스트 2: 추론 ---
    logger.info("\n   --- 테스트 2: 기본 추론 ---")
    test_payload = {
        "prompt": "photo of a cute corgi wearing sunglasses, cinematic lighting, masterpiece, high detail",
        "negative_prompt": "ugly, deformed, blurry, low quality, text, words, letters, signature",
        "steps": 28, "cfg_scale": 7.0, "width": 1024, "height": 1024, "seed": 42
    }
    logger.info(f"   - 요청: {json.dumps(test_payload)}")
    try:
        start_infer_time = time.time()
        response = requests.post(predictions_url, json=test_payload, timeout=400) # 시간 충분히
        response.raise_for_status(); result = response.json()
        end_infer_time = time.time()
        if result and "image_base64" in result:
            logger.info(f"   - 추론 성공! (소요 시간: {end_infer_time - start_infer_time:.2f}초)")
            img_data = base64.b64decode(result["image_base64"])
            img = Image.open(io.BytesIO(img_data)); display(img)
        else: logger.error(f"   - 추론 응답 형식 오류: {result}")
    except Exception as e: logger.error(f"   - 추론 실패: {e}")

    # --- 테스트 3 & 4 & 5 (LoRA 관련) ---
    # (이전 코드와 동일, NEW_LORA_PATH 설정 필요)
    NEW_LORA_PATH = "" # <--- 테스트할 다른 LoRA 경로 설정
    if INITIAL_LORA_WEIGHTS_PATH or NEW_LORA_PATH:
        logger.info(f"\n   --- LoRA 변경 테스트 (NEW_LORA_PATH='{NEW_LORA_PATH}') ---")
        # 1. 언로드 테스트 (초기 LoRA가 있었다면)
        if INITIAL_LORA_WEIGHTS_PATH:
            logger.info("   - LoRA 언로드 시도...")
            try:
                resp_unload = requests.post(unload_lora_url, timeout=60)
                resp_unload.raise_for_status(); logger.info(f"     LoRA 언로드 응답: {resp_unload.json()}")
            except Exception as e: logger.error(f"     LoRA 언로드 실패: {e}")
            time.sleep(2) # 적용 시간

        # 2. 새 LoRA 로드 테스트 (경로가 지정되었다면)
        if NEW_LORA_PATH and os.path.exists(NEW_LORA_PATH):
             logger.info(f"   - 새 LoRA 로드 시도: {NEW_LORA_PATH}")
             try:
                 resp_load = requests.post(load_lora_url, json={"lora_path": NEW_LORA_PATH}, timeout=120)
                 resp_load.raise_for_status(); logger.info(f"     새 LoRA 로드 응답: {resp_load.json()}")
             except Exception as e: logger.error(f"     새 LoRA 로드 실패: {e}")
             time.sleep(2) # 적용 시간

             # 3. 새 LoRA 적용 후 추론 테스트
             logger.info("   - 새 LoRA 적용 후 추론 시도...")
             try:
                 start_infer_time = time.time()
                 response = requests.post(predictions_url, json=test_payload, timeout=400) # 동일 페이로드 사용
                 response.raise_for_status(); result = response.json()
                 end_infer_time = time.time()
                 if result and "image_base64" in result:
                     logger.info(f"   - 새 LoRA 추론 성공! (소요 시간: {end_infer_time - start_infer_time:.2f}초)")
                     img_data = base64.b64decode(result["image_base64"])
                     img = Image.open(io.BytesIO(img_data)); display(img)
                 else: logger.error(f"   - 새 LoRA 추론 응답 형식 오류: {result}")
             except Exception as e: logger.error(f"   - 새 LoRA 적용 후 추론 실패: {e}")
        elif NEW_LORA_PATH:
             logger.warning(f"   - 새 LoRA 경로({NEW_LORA_PATH})가 유효하지 않아 로드 테스트 건너<0xEB><0x9C><0x8D>.")

else:
    logger.warning("⚠️ API 테스트를 건너<0xEB><0x9C><0x8D>니다 (URL 생성 실패 또는 서버 미준비).")


# ==============================================================================
# 7. 서버 정보 및 종료 안내
# ==============================================================================
print("\n" + "="*60)
print("🎉 스크립트 실행 완료 🎉")
print("="*60)
print("="*60)
# server_ready 변수는 코드 블록 4 끝에서 FastAPI 서버 상태에 따라 True 또는 False로 설정됨
# server_ok 대신 server_ready 변수를 직접 사용합니다.

# FastAPI 서버가 성공적으로 준비되었는지 확인
if 'server_ready' in locals() and server_ready: # server_ready 변수가 존재하고 True 인지 확인
    if public_url: print(f"✅ 외부 접속 URL (ngrok): {public_url}")
    else: print(f"⚠️ ngrok URL 생성 실패 또는 Colab 환경 아님.")
    print(f"\n🔗 API 엔드포인트:")
    print(f"  - 추론 (POST) : {target_base_url}/predictions")
    print(f"  - 상태 (GET)  : {target_base_url}/status")
    print(f"  - LoRA 로드(POST): {target_base_url}/load-lora")
    print(f"  - LoRA언로드(POST): {target_base_url}/unload-lora")
    print(f"\n📄 Uvicorn 로그 파일: {UVICORN_LOG_FILE}")
    print("\n🚀 FastAPI 서버와 ngrok 터널(실행된 경우)이 백그라운드에서 실행 중입니다.")
    print("   - API를 계속 사용하려면 이 Colab 노트북 세션을 활성 상태로 유지하세요.")
    print("   - 서버/터널을 중지하려면 아래 8번 셀의 주석을 해제하고 실행하세요.")
else:
    print("\n❌ 서버 시작 또는 모델 준비에 실패했습니다.")
    print("   - 위의 로그를 검토하여 원인을 확인하세요.")
    print("   - 특히 Uvicorn 로그 파일을 확인하는 것이 도움이 될 수 있습니다.")

# ==============================================================================
# 8. 서버 및 터널 종료 (주석 해제 후 실행)
# ==============================================================================
# logger.info("\n⏹️ FastAPI 서버 및 ngrok 터널 종료 중...")
# try:
#     from pyngrok import ngrok
#     logger.info("   - ngrok 터널 종료 시도...")
#     ngrok.kill()
#     logger.info("   - ngrok 프로세스 종료 완료.")
# except ImportError: logger.warning("   - pyngrok 미설치됨.")
# except Exception as e: logger.error(f"   - ngrok 종료 중 오류: {e}")
#
# logger.info("   - Uvicorn 프로세스 종료 시도 (pkill)...")
# # pkill이 항상 성공하는 것은 아님
# result_pkill = subprocess.run(['pkill', '-f', 'uvicorn main:app'], capture_output=True)
# logger.info(f"   - pkill 결과: {result_pkill.returncode}")
# time.sleep(3)
# logger.info("✅ 서버 종료 완료 시도.")
# # Uvicorn 프로세스가 여전히 살아있는지 확인 (선택적)
# result_pgrep = subprocess.run(['pgrep', '-f', 'uvicorn main:app'], capture_output=True)
# if result_pgrep.stdout: logger.warning("   - Uvicorn 프로세스가 여전히 실행 중일 수 있습니다.")


--- Uvicorn 로그 (마지막 20줄) ---
E0000 00:00:1744961699.488144   47255 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744961699.494652   47255 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-18 07:34:59.516337: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO:     Started server process [47255]
INFO:     Waiting for application startup.
INFO:     Lifespan: 애플리케이션 시작 - 모델 로딩...
INFO:     Lifespan: Device=cuda, Dtype=torch.float16
INFO:     Lifespan: 메모리 (로딩 전) Allocated=0.00GB, Reserved=0.00GB

Loading pipeline components...: 100%|████

ERROR:__main__:   - 상태 조회 실패: No connection adapters were found for 'NgrokTunnel: "https://172f-35-247-173-221.ngrok-free.app" -> "http://localhost:9080"/status'
ERROR:__main__:   - 추론 실패: No connection adapters were found for 'NgrokTunnel: "https://172f-35-247-173-221.ngrok-free.app" -> "http://localhost:9080"/predictions'



🎉 스크립트 실행 완료 🎉
✅ 외부 접속 URL (ngrok): NgrokTunnel: "https://172f-35-247-173-221.ngrok-free.app" -> "http://localhost:9080"

🔗 API 엔드포인트:
  - 추론 (POST) : NgrokTunnel: "https://172f-35-247-173-221.ngrok-free.app" -> "http://localhost:9080"/predictions
  - 상태 (GET)  : NgrokTunnel: "https://172f-35-247-173-221.ngrok-free.app" -> "http://localhost:9080"/status
  - LoRA 로드(POST): NgrokTunnel: "https://172f-35-247-173-221.ngrok-free.app" -> "http://localhost:9080"/load-lora
  - LoRA언로드(POST): NgrokTunnel: "https://172f-35-247-173-221.ngrok-free.app" -> "http://localhost:9080"/unload-lora

📄 Uvicorn 로그 파일: /content/uvicorn.log

🚀 FastAPI 서버와 ngrok 터널(실행된 경우)이 백그라운드에서 실행 중입니다.
   - API를 계속 사용하려면 이 Colab 노트북 세션을 활성 상태로 유지하세요.
   - 서버/터널을 중지하려면 아래 8번 셀의 주석을 해제하고 실행하세요.
